**Importing Libraries**

In [2]:
import zipfile
import os
from shutil import copyfile
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from tensorflow.keras.optimizers import RMSprop

**Downloading the Data into a Zip File and Extracting it**

In [3]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


--2020-07-04 06:51:11--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.38.114.234, 2600:1407:a800:1ac::e59, 2600:1407:a800:19e::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.38.114.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M  59.1MB/s    in 16s     

2020-07-04 06:51:28 (47.8 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



**Making Directories for Training and Testing Data**

These directories have subdirectories as Cats and Dogs

In [4]:
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

**Data Preprocessing**

In this we do the following things :


1.   Extract Cat and Dog images from the directories
2.   Split them into training and test data
3.   Shuffle the data
4.   Save them into destination directories created in the previous cell



In [5]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)



CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)


666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


**Checking if the above cell worked by listing the number of images in each directory**

In [6]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))


11250
11250
1250
1250


**Initialising the ImageDataGenerator**

In [7]:
generator = image.ImageDataGenerator(rescale=  1./255)

In [8]:
train_dir = '/tmp/cats-v-dogs/training/'
test_dir = '/tmp/cats-v-dogs/testing/'
train_data_gen = generator.flow_from_directory(train_dir, target_size=(150,150),class_mode='binary',batch_size=100)

Found 22498 images belonging to 2 classes.


**Building the Neural Network**

In [13]:
model = Sequential()
model.add(Conv2D(16,(3,3),activation = 'relu', input_shape=(150,150,3)))
model.add(MaxPool2D(2,2))

model.add(Conv2D(32,(3,3),activation = 'relu', input_shape=(150,150,3)))
model.add(MaxPool2D(2,2))

model.add(Conv2D(64,(3,3),activation = 'relu', input_shape=(150,150,3)))
model.add(MaxPool2D(2,2))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)            

In [14]:
model.compile(optimizer=RMSprop(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

**Training the network and evaluating on the validation data**

In [16]:
model.fit(train_data_gen,epochs=5)

Epoch 1/5
111/225 [=============>................] - ETA: 39s - loss: 0.4799 - accuracy: 0.7705

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

225/225 [==============================] - 80s 355ms/step - loss: 0.4695 - accuracy: 0.7781
Epoch 2/5
225/225 [==============================] - 80s 356ms/step - loss: 0.4014 - accuracy: 0.8169
Epoch 3/5
225/225 [==============================] - 80s 358ms/step - loss: 0.3372 - accuracy: 0.8535
Epoch 4/5
225/225 [==============================] - 79s 353ms/step - loss: 0.2686 - accuracy: 0.8851
Epoch 5/5
225/225 [==============================] - 80s 355ms/step - loss: 0.1992 - accuracy: 0.9202


In [18]:
validation_generator = image.ImageDataGenerator(rescale=1./255)

In [19]:
test_data_gen = validation_generator.flow_from_directory(test_dir,target_size=(150,150),class_mode='binary',batch_size=20) 

Found 2500 images belonging to 2 classes.


In [20]:
model.evaluate(test_data_gen)

125/125 [==============================] - 9s 72ms/step - loss: 0.4338 - accuracy: 0.8368


[0.43376415967941284, 0.8367999792098999]

**We received an accuracy of 92% on training data and 83% on validation data**